In [1]:
###Import packages
import networkx as nx 
import osmnx as ox
import warnings
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, LineString, Polygon
import itertools
from tqdm import tqdm
from pyproj import Proj, transform

warnings.filterwarnings("ignore")

---
> #### Raw data
1. 기본 설명 : 전국 사고 데이터
2. 데이터 수집 기관 : 경찰청
3. 기간 : 2016.01~2019.12(4년)
4. data 갯수 : 884000개
5. features 갯수 : 90개

- 미사용 컬럼
    - ACC_YM, ORG_CD, ACC_NO, HIT_BJD_CD, ACC_PLC, ACC_STU, GRS_X_CRD, GRS_Y_CRD, POL_RPT_YN, WEA_STA_CD, BIG_ACC_YN, DEATH_CNT, SERI_CNT, SLTWD_CNT,
    WND_CNT, DOBJ_CNT, DMG_AMT, OLD_CHLD_INC_YN, DRK_ACC_YN, RUN_ACC_YN, UNLIC_ACC_YN, HIT_ACC_CRD, LINK_X_CRD, LINK_Y_CRD, IMG_DIR_NM, TWV_ACC_YN,
    HAM_TWV_ACC_YN, VCT_TWV_ACC_YN, MOTR_ACC_YN, HAM_MOTR_ACC_YN, VCT_MOTR_ACC_YN, BCL_ACC_YN, HAM_BCL_ACC_YN, VCT_BCL_ACC_YN, CT_EQUIP_ACC_YN,
    HAM_CT_EQUIP_ACC_YN, VCT_CT_EQUIP_ACC_YN, AC_EQUIP_ACC_YN, HAM_AC_EQUIP_ACC_YN, VCT_AC_EQUIP_ACC_YN, WKR_ACC_YN, HAM_WKR_ACC_YN, VCT_WKR_ACC_YN,
    CHLD_ACC_YN, HAM_CHLD_ACC_YN, VCT_CHLD_ACC_YN, OLD_ACC_YN, HAM_OLD_ACC_YN, VCT_OLD_ACC_YN, MEDLAR_ACC_YN, MEDLAR_PSGT_YN, MEDLAR_TRK_YN,
    CAR_USE_CD, DRV_LIC_CRR, DRV_LIC_CLSF_CD, FORN_YN_CD, NODE_ID, LINK_ID, RD_NO, RD_NM, RD_PAV_YN, RD_MED_YN, TOT_LANE_CNT, RD_GRD_CD, LINK_CLSF_CD,
    ESTB_CLSF_CD, LMT_SPD, BUS_CNT_CAR_RD_YN, ONE_TRF_YN, LOT_NO, DWG_228, ETL_RGST_ID, ETL_REG_DT

- 사용 컬럼
    - ACC_YMD, ACC_TME, DAY_CD, BJD_CD, ACC_TYP_CD, LAW_VIO_CD, TM_X_CRD, TM_Y_CRD, RD_CLS_CD, RD_TYP_CD, ACC_GRD_CD, CAR_CLSF_CD, SEX_CD, AGE_CD, ACC_TYP_MAP_CD, ACC_FTR_CD

---
> #### Daejeon truck accident data
1. 기본 설명 : 대전 화물 사고 데이터
2. 데이터 수집 기관 : 경찰청
3. 기간 : 2016.01~2019.12(4년)
4. data 갯수 : 3076개
5. features 갯수 : 16개

- 컬럼 설명 
    - ACC_YMD : 년/월/일
    - ACC_TME : 시/분
    - DAY_CD : 요일
    - BJD_CD : 법정동 코드
    - ACC_TYP_CD : 사고 유형 코드
    - LAW_VIO_CD : 법규 위반 코드
    - TM_X_CRD : TM x좌표
    - TM_Y_CRD : TM y좌표
    - RD_CLS_CD : 도로 종류 코드
    - RD_TYP_CD : 도로 형태 코드
    - ACC_GRD_CD : 사고 등급 코드
    - CAR_CLSF_CD : 차량 종별 코드
    - SEX_CD : 성별 코드 
    - AGE_CD : 나이 코드
    - ACC_TYP_MAP_CD : 사고 형태 지도 코드 
    - ACC_FTR_CD : 사고 요인 코드

In [2]:
###Load Datasets (ACC_INFO.csv : 사고 데이터, 사고_코드정보.xls: 사고 데이터 컬럼 정보)
acc_data = pd.read_csv("/home/yh_zoo/data/ACC_INFO.csv", sep="\t")
code_info = pd.read_excel("/home/yh_zoo/data/사고_코드정보.xls")

In [3]:
def prepared_data(data):
    ###truck filter
    data = data.loc[data["CAR_CLSF_CD"] == 3]   
    ###고속도로 사고 제외
    data = data.loc[data.RD_CLS_CD != 6.0]
    ###Daejeon region filter
    data = data.loc[(data["BJD_CD"] >= 3000000000) & (data["BJD_CD"] < 3023012700)].reset_index(drop=True)
    ###features filter
    data = data[["ACC_YMD", "ACC_TME", "DAY_CD", "BJD_CD", "ACC_TYP_CD", "LAW_VIO_CD", "TM_X_CRD","TM_Y_CRD", "RD_CLS_CD",
                        "RD_TYP_CD", "ACC_GRD_CD", "CAR_CLSF_CD", "SEX_CD", "AGE_CD", "ACC_TYP_MAP_CD", "ACC_FTR_CD"]]
    #coordinate transformation
    def transform_utmk_to_w84(df):
        proj_UTMK = Proj(init='epsg:5179')
        proj_WGS84 = Proj(init='epsg:4326')
        return pd.Series(transform(proj_UTMK, proj_WGS84, df['TM_X_CRD'], df['TM_Y_CRD']), index=['x', 'y'])
    data[['TM_Y_CRD','TM_X_CRD']] = data.apply(transform_utmk_to_w84, axis=1)
    
    return data

acc_data  = prepared_data(acc_data)

**도로 규모별 구분**

1. 광로
    - 1류 : 폭 70m 이상인 도로
    - 2류 : 폭 50m 이상 70m 미만인 도로
    - 3류 : 폭 40m 이상 50m 미만인 도로

2. 대로
    - 1류 : 폭 35m 이상 40m 미만인 도로
    - 2류 : 폭 30m 이상 35m 미만인 도로
    - 3류 : 폭 25m 이상 30m 미만인 도로

3. 중로
    - 1류 : 폭 20m 이상 25m 미만인 도로
    - 2류 : 폭 15m 이상 20m 미만인 도로
    - 3류 : 폭 12m 이상 15m 미만인 도로

4. 소로
    - 1류 : 폭 10m 이상 12m 미만인 도로
    - 2류 : 폭 8m 이상 10m 미만인 도로
    - 3류 : 폭 8m 미만인 도로



**도로 기능별 구분**	
- 주 간 선 도 로
- 보조 간선 도로
- 집 산 도 로
- 국 지 도 로

In [6]:
edges  = pd.read_pickle("gdf_edges.pkl")
nodes = pd.read_pickle("gdf_nodes.pkl")

---
- 도로 대분류 

In [8]:
### 국도
primary = edges.loc[list(map(lambda data: "primary" in data, edges.highway))] #0 대로

secondary =  edges.loc[list(map(lambda data: "secondary" in data, edges.highway))] #0 주요 간선 도로

tertiary = edges.loc[list(map(lambda data: "tertiary" in data, edges.highway))] #0 도시의 지방 중심지를 연결하는 도로

In [11]:
residential = edges.loc[list(map(lambda data: "residential" in data, edges.highway))] #1 주거 지역의 도로

living_street = edges.loc[list(map(lambda data: "living_street" in data, edges.highway))] #1 보행자 우선도로

road = edges.loc[list(map(lambda data: "road" in data, edges.highway))] #1 미분류 도로

service = edges.loc[list(map(lambda data: "service" in data, edges.highway))] #1 건물, 주유소, 해변, 캠프장, 산업 단지, 비즈니스 파크 (사도)

unclassified = edges.loc[list(map(lambda data: "unclassified" in data, edges.highway))] #1 공공 접근 도로

In [13]:
###대전광역시_법정경계 Data
places = ox.geocode_to_gdf(["대전,대한민국"])
places = ox.project_gdf(places)

#lat, lon
latitude = 36.342112; longitude = 127.387424

#기본 지도 정의
m = folium.Map(location=[latitude, longitude],
               zoom_start=11)

#법정경계 표시
folium.Choropleth(geo_data=places.geometry,
                  fill_color="white",
                  ).add_to(m)
    
#Links
######################################
folium.Choropleth(
    primary.geometry,
    line_weights=3,
    line_color="red",   
    ).add_to(m) 

folium.Choropleth(
    secondary.geometry,
    line_weights=3,
    line_color="red",   
    ).add_to(m) 

folium.Choropleth(
    tertiary.geometry,
    line_weights=3,
    line_color="red",   
    ).add_to(m) 
####################################
folium.Choropleth(
    residential.geometry,
    line_weights=3,
    line_color="blue",   
    ).add_to(m) 

folium.Choropleth(
    living_street.geometry,
    line_weights=3,
    line_color="blue",   
    ).add_to(m) 

folium.Choropleth(
    road.geometry,
    line_weights=3,
    line_color="blue",   
    ).add_to(m) 

folium.Choropleth(
    service.geometry,
    line_weights=3,
    line_color="blue",   
    ).add_to(m)

folium.Choropleth(
    unclassified.geometry,
    line_weights=3,
    line_color="blue",   
    ).add_to(m)  

m.save("map.html")